In [1]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [2]:
import numpy as np
import pandas as pd
from typing import  List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation
from sklearn.model_selection import train_test_split
import nltk
from collections import Counter
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, T5EncoderModel, AutoModelForSequenceClassification, pipeline
import pandas as pd
import time
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer

2025-05-10 02:26:59.423484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746844019.611232      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746844019.663105      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Загружаем новости с сильными ветрами

In [3]:
wind = pd.read_excel('/kaggle/input/wind-bof/wind_bof_dis.xlsx')

In [4]:
wind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12734 entries, 0 to 12733
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Unnamed: 0              12734 non-null  int64         
 1   _ID                     12734 non-null  object        
 2   Event_ID                11005 non-null  object        
 3   News_link               12734 non-null  object        
 4   Text                    12734 non-null  object        
 5   News_length             12734 non-null  int64         
 6   Количество              12734 non-null  int64         
 7   News_source             12734 non-null  object        
 8   News_type               12734 non-null  object        
 9   Location_ATD4           12679 non-null  object        
 10  Location_other          11201 non-null  object        
 11  Location_geocoded       12734 non-null  object        
 12  Location_geocoded_bool  12734 non-null  bool  

### С помощью модели [Frida](https://huggingface.co/ai-forever/FRIDA) производится верификация определенного в БД в части с сильными ветрами типе события (или типах) по тексту новости

In [7]:
# Начало отсчета времени
start_time = time.time()

# Загрузка модели с CLS pooling
model = SentenceTransformer("ai-forever/FRIDA")

# Пример DataFrame (замените на ваш DataFrame)
df = wind.copy()

# Функция для верификации
def to_verify(row):
    text = row['Text']
    phenomena = row['Phenomena']
    
    # Получение эмбеддингов
    text_embedding = model.encode(text,
                                  prompt_name='categorize_topic',
                                  show_progress_bar=False)
    phenomena_embedding = model.encode(phenomena,
                                       prompt_name='categorize_topic',
                                       show_progress_bar=False)
    
    # Вычисление косинусного сходства
    similarity_score = text_embedding @ phenomena_embedding.T
    return similarity_score

# Применение функции к каждой строке DataFrame
df['score_Phenomena'] = df.apply(to_verify, axis=1)

# Завершение отсчета времени
end_time = time.time()
execution_time = end_time - start_time
print(f"\nВремя выполнения: {execution_time // 60:.2f} минут")


Время выполнения: 21.00 минут


In [8]:
df['score_Phenomena'].describe()

count    12734.000000
mean         0.884728
std          0.049493
min          0.708113
25%          0.854804
50%          0.896963
75%          0.923907
max          0.971253
Name: score_Phenomena, dtype: float64

### С помощью модели [Frida](https://huggingface.co/ai-forever/FRIDA) производится верификация типа ОПЯ - сильный ветер в части БД с сильными ветрами

In [5]:
# Начало отсчета времени
start_time = time.time()

# Загрузка модели с CLS pooling
model = SentenceTransformer("ai-forever/FRIDA")

# Пример DataFrame (замените на ваш DataFrame)
df = wind.copy()

# Функция для верификации
def to_verify(row):
    text = row['Text']
    phenomena = row['Phenomena']
    
    # Получение эмбеддингов
    text_embedding = model.encode(text,
                                  prompt_name='categorize_topic',
                                  show_progress_bar=False)
    phenomena_embedding = model.encode('сильный ветер',
                                       prompt_name='categorize_topic',
                                       show_progress_bar=False)
    
    # Вычисление косинусного сходства
    similarity_score = text_embedding @ phenomena_embedding.T
    return similarity_score

# Применение функции к каждой строке DataFrame
df['score_wind'] = df.apply(to_verify, axis=1)

# Завершение отсчета времени
end_time = time.time()
execution_time = end_time - start_time
print(f"\nВремя выполнения: {execution_time // 60:.2f} минут")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/509 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/823 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/5.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]


Время выполнения: 22.00 минут


In [6]:
df['score_wind'].describe()

count    12734.000000
mean         0.882092
std          0.038387
min          0.701751
25%          0.852497
50%          0.895564
75%          0.911722
max          0.951818
Name: score_wind, dtype: float64

### С помощью модели [bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli) производится верификация определенного в БД в части с сильными ветрами типе события (или типах) по тексту новости

In [15]:
# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

start_time = time.perf_counter()

# Загрузка модели и токенизатора
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').to(device)
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

# Пример DataFrame (замените на ваш DataFrame)
res_ph = wind[['Text', 'Phenomena']]

def to_verify_bart(row):
    label = row['Phenomena']
    premise = row['Text']
    hypothesis = f'This example is {label}.'
    
    # Токенизация и перемещение на устройство
    x = tokenizer.encode(premise, hypothesis, return_tensors='pt', truncation=True).to(device)
    
    # Получение логитов
    logits = nli_model(x)[0]
    
    # Вычисление вероятности
    entail_contradiction_logits = logits[:, [0, 2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs[:, 1]
    return prob_label_is_true.item()  # Возвращаем скалярное значение

# Применение функции к каждой строке DataFrame
res_ph['wind_score'] = res_ph.apply(to_verify_bart, axis=1)

end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"Время выполнения: {round(execution_time / 60, 2)} мин")


Время выполнения: 27.26 мин


/tmp/ipykernel_31/4216801656.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_ph['wind_score'] = res_ph.apply(to_verify_bart, axis=1)


In [16]:
res_ph['wind_score'].describe()

count    12734.000000
mean         0.775066
std          0.259712
min          0.001351
25%          0.707024
50%          0.888475
75%          0.952751
max          0.999602
Name: wind_score, dtype: float64

### С помощью модели [bart-large-mnli](https://huggingface.co/facebook/bart-large-mnli) производится верификация типа ОПЯ - сильный ветер в части БД с сильными ветрами

In [13]:
# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

start_time = time.perf_counter()

# Загрузка модели и токенизатора
nli_model = AutoModelForSequenceClassification.from_pretrained('facebook/bart-large-mnli').to(device)
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-mnli')

# Пример DataFrame (замените на ваш DataFrame)
res = wind[['Text']]

def to_verify_bart(text):
    label = 'сильный ветер'
    premise = text
    hypothesis = f'This example is {label}.'
    
    # Токенизация и перемещение на устройство
    x = tokenizer.encode(premise, hypothesis, return_tensors='pt', truncation=True).to(device)
    
    # Получение логитов
    logits = nli_model(x)[0]
    
    # Вычисление вероятности
    entail_contradiction_logits = logits[:, [0, 2]]
    probs = entail_contradiction_logits.softmax(dim=1)
    prob_label_is_true = probs[:, 1]
    return prob_label_is_true.item()  # Возвращаем скалярное значение

# Применение функции к каждой строке DataFrame
res['wind_score'] = res['Text'].apply(to_verify_bart)

end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"Время выполнения: {round(execution_time / 60, 2)} мин")


Время выполнения: 27.11 мин


/tmp/ipykernel_31/1242020041.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res['wind_score'] = res['Text'].apply(to_classify)


In [14]:
res['wind_score'].describe()

count    12734.000000
mean         0.887946
std          0.146392
min          0.014956
25%          0.867059
50%          0.934340
75%          0.973129
max          0.999676
Name: wind_score, dtype: float64

- В случае с `FRIDA` скоры у сильного ветра и у прописанных в БД ОПЯ напротив новости типе (типах) явления примерно совпадают - медиана `0.90`
- В случае с `bart-large-mnli` скоры у сильного ветра и у прописанных в БД ОПЯ различаются, в первом случае медиана = `0.93`, во втором - `0.89`